In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [172]:
import pandas as pd
import numpy as np
from nltk.corpus import wordnet as wn
import gensim 
from gensim.models import Word2Vec 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import matplotlib.pyplot as plt

In [3]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [155]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english")) | {'due', 'unspecified', 'type', 'nan'}
stop_words.remove('with')

In [123]:
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

In [152]:
df_train = np.asarray(pd.read_csv("data/diabetes/diab_train.csv", skiprows=1, header=None))
df_val = np.asarray(pd.read_csv("data/diabetes/diab_validation.csv", skiprows=1, header=None))
df_test = np.asarray(pd.read_csv("data/diabetes/diab_test.csv", skiprows=1, header=None))

In [162]:
X = np.hstack([df_train[:,:-4], df_train[:,-3:]])
X_val = np.hstack([df_val[:,:-4], df_val[:,-3:]])
X_test = np.hstack([df_test[:,:-4], df_test[:,-3:]])
Y = df_train[:,-4]
Y_val = df_val[:,-4]
Y_test = df_test[:,-4]

In [232]:
def diff(corpus, sample):
    return np.linalg.norm(corpus-sample, axis=1)

In [163]:
def sanitize(doc):
    return ' '.join([lem.lemmatize(w) for w in tokenizer.tokenize(str(doc)) if w not in stop_words])

In [210]:
diagnoses = []

for x in X[:, -3:]:
    for i in range(x.size):
        diagnoses.append(sanitize(x[i]))
        
for x in X_val[:, -3:]:
    for i in range(x.size):
        diagnoses.append(sanitize(x[i]))
        
for x in X_test[:, -3:]:
    for i in range(x.size):
        diagnoses.append(sanitize(x[i]))

In [228]:
corpus = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(diagnoses)]
embedder = Doc2Vec(corpus, window=3, min_count=1, workers=-1)

In [229]:
embedded_diag = []
for d in set(diagnoses):
    embedded_diag.append(embedder.infer_vector(d.split()))
embedded_diag = np.asarray(embedded_diag)

In [235]:
X_emb = []
for x in X[:, -3:]:
    t = []
    for i in range(x.size):
        t.append(diff(embedded_diag, embedder.infer_vector(str(x[i]).split())))
    X_emb.append(t)

AttributeError: 'float' object has no attribute 'split'